In [113]:
import time
import itertools
from typing import List, Tuple
from sympy import *
import numpy as np
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.aqua.operators import SummedOp, PauliOp
from qiskit.aqua.operators import I, X, Y, Z

from qiskit_nature.problems.sampling.folding import LatticeFoldingProblem

In [114]:
N = 5
side_chain = [0]*N
lambda_back = 10

In [115]:
lf = LatticeFoldingProblem()

In [116]:
lf.pauli_op()

number of qubits required for conformation:  3
10  distances created
number of qubits required for contact :  0
total number of qubits required : 3
number of terms in the hamiltonian :  4
Hamiltonian:  -5*\sigma^z_{5}*\sigma^z_{7}*\sigma^z_{8}/2 + 5*\sigma^z_{5}*\sigma^z_{7}/2 - 5*\sigma^z_{8}/2 + 5/2
mask at term 0 is [0. 0. 0.]
mask at term 1 is [0. 0. 1.]
mask at term 2 is [1. 1. 0.]
mask at term 3 is [1. 1. 1.]
pauli_list:  [array([-2.5, Pauli('ZZZ')], dtype=object), array([2.5, Pauli('IZZ')], dtype=object), array([-2.5, Pauli('ZII')], dtype=object), array([2.5, Pauli('III')], dtype=object)]


/home/davindratulsi/Desktop/qiskit-nature-fork/qiskit-nature/qiskit_nature/problems/sampling/folding/folding_qubit_op_builder.py:848: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pauli_list = list(np.flip(np.array(pauli_list), axis=0))


SummedOp([PauliOp(Pauli('ZZZ'), coeff=-2.5), PauliOp(Pauli('IZZ'), coeff=2.5), PauliOp(Pauli('ZII'), coeff=-2.5), PauliOp(Pauli('III'), coeff=2.5)], coeff=1.0, abelian=False)

## Test with replacing dictionaries with numpy arrays, build geometrical constraint contribution to Hamiltonian (prevent chain from folding back on itself), H_back

Ideally, we would like to remove/replace dictionaries of Pauli terms as well as the symbolic math notation. The symbolic math notation gives  a nice visual representation of the Hamiltonian, however, subsitution methods to simplify the Hamiltonian are slow. Replace symbolic Paulis with operators?

In [126]:
def _create_pauli_for_conf(N):
    """
    Create dictionary of Pauli operators that define the conformation
    of a peptide fold in an Ising spin glass model. The Pauli operator,
    sigma^z_{i} is related to the qubit register, q_i as follows:
    sigma^z_{i} = 1 - 2*q_i.

    Args:
        N: Number of total beads in peptide

    Returns:
        pauli_conf: Dictionary of Pauli Z-matrices in symbolic
                    math notation for all 2*(N-1) turns.
                    Note that each turn consists of two Pauli
                    terms, pauli_conf[i][0] and pauli_conf[i][1]
                    corresponding to the backbone and side chain
                    beads respectively.
    """
    num_turns = 2*(N - 1)
    pauli_conf = np.zeros((num_turns, 2), dtype=object)
#     pauli_conf = dict()
    for i in range(num_turns):
#         pauli_conf[i] = dict()
        pauli_conf[i][0] = symbols("\sigma^z_{}".format({i}))
        pauli_conf[i][1] = symbols("\sigma^z_"+"{" + "{}".format({i}) + "^{(1)}"+"}")
    return pauli_conf

def _create_qubits_for_conf(pauli_conf):
    """
    Create conformation qubits based on the Pauli Z operators,
    for backbone and side chain beads. Conversely as above,
    this transformation moves from the spin (-1,1) Hamiltonian
    to the qubit (0,1) Hamiltonian. That is, qubit, q_i is
    transformed as (1 - sigma^z_{i})/2

    Args:
        pauli_conf: Dictionary of Pauli Z-matrices in
                    symbolic math notation

    Returns:
        qubits: Dictionary of qubits in symbolic notation.
                Note that each turn consists of two qubit
                registers, qubits[i][0] and qubits[i][1]
                corresponding to the backbone and side chain
                beads respectively.
    """
    # qubits = dict()
    qubits = np.zeros(pauli_conf.shape, dtype=object)
    num_turns = qubits.shape[0]
    for i in range(num_turns):
    # for i in range(1, len(pauli_conf) + 1):
    #     qubits[i] = dict()
        qubits[i][0] = (1 - pauli_conf[i][0])/2
        qubits[i][1] = (1 - pauli_conf[i][1])/2
    return qubits


def _create_indic_turn(N, side_chain, qubits):
    """
    Creates indicator functions that specify the axis chosen for a
    corresponding turn. Here, each turn, i (from 1 to N-1) is (densely)
    coded on two qubits registers, located at 2i - 1 and 2i.
    Each function returned is of the form, indica(i),
    which returns 1 if axis, a = 0,1,2,3 is chosen at turn i.

    Args:
        N: Number of total beads in peptide
        side_chain: List of side chains in peptide
        qubits: Dictionary of conformation qubits in symbolic notation

    Returns:
        (indic_0, indic_1, indic_2, indic_3, num_qubits): Turn indicators for the four axes,
                                                      0,1,2,3. Note, as in the pauli and qubit
                                                      conformation notations,
                                                      indic_a[i][0] and indic_a[i][1] refer to
                                                      backbone and side chain respectively.
    """
    if len(side_chain)!= N:
        raise Exception('size of side_chain list is not equal to N')
    # indic_0, indic_1, indic_2, indic_3 = dict(), dict(), dict(), dict()
    # for i in range(1, N):
    #     indic_0[i] = dict()
    #     indic_1[i] = dict()
    #     indic_2[i] = dict()
    #     indic_3[i] = dict()
    num_turns = N - 1 
    indic_0 = np.zeros((num_turns, 2), dtype=object)
    indic_1 = np.zeros((num_turns, 2), dtype=object)
    indic_2 = np.zeros((num_turns, 2), dtype=object)
    indic_3 = np.zeros((num_turns, 2), dtype=object)
    r_conf = 0
    for i in range(num_turns):
#     for i in range(1, N):   # There are N-1 turns starting at turn 1
        for m in range(2):
            if m == 1:
                if side_chain[i - 1] == 0:
                    continue
                else:
                    pass
            indic_0[i][m] = (1 - qubits[2*i - 1][m])*(1 - qubits[2*i][m])
            indic_1[i][m] = qubits[2*i][m]*(qubits[2*i][m] - qubits[2*i - 1][m])
            indic_2[i][m] = qubits[2*i - 1][m]*(qubits[2*i - 1][m]-qubits[2*i][m])
            indic_3[i][m] = qubits[2*i - 1][m]*qubits[2*i][m]
            r_conf += 1
    num_qubits = 2*r_conf - 5
    print('number of qubits required for conformation: ', num_qubits)
    return indic_0, indic_1, indic_2, indic_3, num_qubits

def _check_turns(i, p, j, s,
                 indic0, indic1, indic2,
                 indic3, pauli_conf):
    """
    Checks if consecutive turns are along the same axis. Specifically,
    the function is the summation over all axes, a = 0,1,2,3, of the
    product of turn indicators, indica(i)*indica(j) for turns i and j.


    Args:
        i: Backbone bead at turn i
        j: Backbone bead at turn j (j > i)
        p: Side chain on backbone bead j
        s: Side chain on backbone bead i
        indic0: Turn indicator for axis 0
        indic1: Turn indicator for axis 1
        indic2: Turn indicator for axis 2
        indic3: Turn indicator for axis 3
        pauli_conf: Dictionary of conformation Pauli operators in symbolic notation

    Returns:
        t_ij: Production of turn indicators in symbolic notation
    """
#     t_ij = _simplify(pauli_conf, indic0[i][p]*indic0[j][s] + indic1[i][p]*indic1[j][s] + \
#                      indic2[i][p]*indic2[j][s] +indic3[i][p]*indic3[j][s])
    t_ij = indic0[i][p]*indic0[j][s] + indic1[i][p]*indic1[j][s] + \
           indic2[i][p]*indic2[j][s] +indic3[i][p]*indic3[j][s]   
    return t_ij

In [127]:
def _simplify(pauli_conf, x):
    """
    Simplifies a Symbolic Hamiltonian term by reducing the number
    of Pauli-Z operators and subsituting pre-defined
    values for turns. In specific, all even powers of Pauli 
    terms in the Symbolic Hamiltonian term are substituted 
    for a value of 1 since, (sigma^z_{i})**2 = I (identity).
    Additionally, the first two turns are fixed to be (in binary)
    01 and 00, which translate to 1,-1 and 1,1, respectively in
    spin variables. With no side chain on the 2nd bead, we further
    fix the value of the 6th qubit, q_6 = 1 (-1 for the corresponding
    Pauli).

    Args:
        pauli_conf: Dictionary of Pauli operators
        x: Symbolic Hamiltonian term to be simplifed

    Returns:
        x: Simplified Symbolic Hamiltonian
    """
    first_binaries = [1, -1, 1, 1, 0, -1] # hardcoded, fix this
    num_terms = pauli_conf.shape[0]
    if x == 0:
        return 0
    else:
        x = x.expand()
        x = x.subs({pauli_conf[k][0]: first_binaries[k-1] for k in [1, 2, 3, 4, 6]})
#         for m in range(4, 1, -1):
        for m in [4, 3, 2]:
#             x = x.subs({pauli_conf[k][0]**m: (pauli_conf[k][0])**(m%2) for k in pauli_conf})
#             x = x.subs({pauli_conf[k][1]**m: (pauli_conf[k][1])**(m%2) for k in pauli_conf})
            x = x.subs({pauli_conf[k][0]**m: (pauli_conf[k][0])**(m%2) for k in range(num_terms)})
            x = x.subs({pauli_conf[k][1]**m: (pauli_conf[k][1])**(m%2) for k in range(num_terms)})
#             x = x.subs({pauli_conf[k][0]: first_binaries[k-1] for k in [1, 2, 3, 4, 6]})
    return x


def _create_H_back(N, lambda_back, indic_0,
                   indic_1, indic_2, indic_3,
                   pauli_conf):
    """
    Creates Hamiltonian that imposes the geometrical constraint wherein consecutive turns
    (N - 1) along the same axis are penalized by a factor, lambda_back. Note,
    that the first two turns are omitted.

    Args:
        N: Number of total beads in peptide
        lambda_back: Constrain that penalizes turns along the same axis
        indic_0: Turn indicator for axis 0
        indic_1: Turn indicator for axis 1
        indic_2: Turn indicator for axis 2
        indic_3: Turn indicator for axis 3
        pauli_conf: Dictionary of conformation Pauli operators in symbolic notation

    Returns:
        H_back: Contribution to Hamiltonian in symbolic notation that penalizes
                consecutive turns along the same axis
    """
    H_back = 0
#     for i in range(1, N - 1):
    for i in range(N - 2):
        H_back += lambda_back*_check_turns(i, 0, i + 1, 0,
                                           indic_0, indic_1, indic_2, indic_3, pauli_conf)
#     H_back = _simplify(pauli_conf, H_back)
    return H_back

In [121]:
start_time = time.time()
pauli_conf = _create_pauli_for_conf(N)
qubits = _create_qubits_for_conf(pauli_conf)
indic_0, indic_1, indic_2, indic_3, num_qubits = _create_indic_turn(N, side_chain, qubits)
H_back = _create_H_back(N, lambda_back, indic_0, indic_1, indic_2, indic_3, pauli_conf)
H_back_simplified = _simplify(pauli_conf, H_back)
print("--- %s seconds ---" % (time.time() - start_time))

number of qubits required for conformation:  3
--- 0.22611165046691895 seconds ---


In [124]:
H_back

10*(1/2 - \sigma^z_{0}/2)*(1/2 - \sigma^z_{1}/2)*(1/2 - \sigma^z_{2}/2)*(1/2 - \sigma^z_{7}/2) + 10*(1/2 - \sigma^z_{0}/2)*(1/2 - \sigma^z_{2}/2)*(-\sigma^z_{0}/2 + \sigma^z_{7}/2)*(\sigma^z_{1}/2 - \sigma^z_{2}/2) + 10*(1/2 - \sigma^z_{1}/2)*(1/2 - \sigma^z_{2}/2)*(1/2 - \sigma^z_{3}/2)*(1/2 - \sigma^z_{4}/2) + 10*(1/2 - \sigma^z_{1}/2)*(1/2 - \sigma^z_{3}/2)*(-\sigma^z_{1}/2 + \sigma^z_{2}/2)*(-\sigma^z_{3}/2 + \sigma^z_{4}/2) + 10*(1/2 - \sigma^z_{1}/2)*(1/2 - \sigma^z_{7}/2)*(\sigma^z_{0}/2 - \sigma^z_{7}/2)*(-\sigma^z_{1}/2 + \sigma^z_{2}/2) + 10*(1/2 - \sigma^z_{2}/2)*(1/2 - \sigma^z_{4}/2)*(\sigma^z_{1}/2 - \sigma^z_{2}/2)*(\sigma^z_{3}/2 - \sigma^z_{4}/2) + 10*(1/2 - \sigma^z_{3}/2)*(1/2 - \sigma^z_{4}/2)*(1/2 - \sigma^z_{5}/2)*(1/2 - \sigma^z_{6}/2) + 10*(1/2 - \sigma^z_{3}/2)*(1/2 - \sigma^z_{5}/2)*(-\sigma^z_{3}/2 + \sigma^z_{4}/2)*(-\sigma^z_{5}/2 + \sigma^z_{6}/2) + 10*(1/2 - \sigma^z_{4}/2)*(1/2 - \sigma^z_{6}/2)*(\sigma^z_{3}/2 - \sigma^z_{4}/2)*(\sigma^z_{5}/2 - \sigma^

In [125]:
H_back_simplified

-5*\sigma^z_{0}*\sigma^z_{7}/2 - 5*\sigma^z_{0}/2 + 5*\sigma^z_{7}/2 + 5/2